# Building the dataset of research papers

The [Entrez](http://biopython.org/DIST/docs/api/Bio.Entrez-module.html) module, a part of the [Biopython](http://biopython.org/) library, will be used to interface with [PubMed](http://www.ncbi.nlm.nih.gov/pubmed).<br>
You can download Biopython from [here](http://biopython.org/wiki/Download).

In this notebook we will be covering several of the steps taken in the [Biopython Tutorial](http://biopython.org/DIST/docs/tutorial/Tutorial.html), specifically in [Chapter 9  Accessing NCBI’s Entrez databases](http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc109).

In [1]:
%pip install biopython

Note: you may need to restart the kernel to use updated packages.


In [2]:
from Bio import Entrez

# NCBI requires you to set your email address to make use of NCBI's E-utilities
Entrez.email = "r.m.siebes@vu.nl"

The datasets will be saved as serialized Python objects, compressed with bzip2.
Saving/loading them will therefore require the [pickle](http://docs.python.org/3/library/pickle.html) and [bz2](http://docs.python.org/3/library/bz2.html) modules.

In [65]:
import pickle, bz2, os, zipfile

## EInfo: Obtaining information about the Entrez databases

In [4]:
# accessing extended information about the PubMed database
pubmed = Entrez.read( Entrez.einfo(db="pubmed"), validate=False )[u'DbInfo']

# list of possible search fields for use with ESearch:
search_fields = { f['Name']:f['Description'] for f in pubmed["FieldList"] }

In search_fields, we find 'TIAB' ('Free text associated with Abstract/Title') as a possible search field to use in searches.

In [5]:
search_fields

{'ALL': 'All terms from all searchable fields',
 'UID': 'Unique number assigned to publication',
 'FILT': 'Limits the records',
 'TITL': 'Words in title of publication',
 'MESH': 'Medical Subject Headings assigned to publication',
 'MAJR': 'MeSH terms of major importance to publication',
 'JOUR': 'Journal abbreviation of publication',
 'AFFL': "Author's institutional affiliation and address",
 'ECNO': 'EC number for enzyme or CAS registry number',
 'SUBS': 'CAS chemical name or MEDLINE Substance Name',
 'PDAT': 'Date of publication',
 'EDAT': 'Date publication first accessible through Entrez',
 'VOL': 'Volume number of publication',
 'PAGE': 'Page number(s) of publication',
 'PTYP': 'Type of publication (e.g., review)',
 'LANG': 'Language of publication',
 'ISS': 'Issue number of publication',
 'SUBH': 'Additional specificity for MeSH term',
 'SI': 'Cross-reference from publication to other databases',
 'MHDA': 'Date publication was indexed with MeSH terms',
 'TIAB': 'Free text associa

## ESearch: Searching the Entrez databases

To have a look at the kind of data we get when searching the database, we'll perform a search for papers authored by Haasdijk:

In [6]:
example_authors = ['Haasdijk E']
example_search = Entrez.read( Entrez.esearch( db="pubmed", term=' AND '.join([a+'[AUTH]' for a in example_authors]) ) )
example_search

{'Count': '41', 'RetMax': '20', 'RetStart': '0', 'IdList': ['39325889', '38631313', '33501027', '33501026', '33500899', '29311830', '28513205', '28513201', '28323435', '28140628', '26933487', '24977986', '24901702', '24852945', '24708899', '24252306', '23580075', '23144668', '22174697', '22154920'], 'TranslationSet': [{'From': 'Haasdijk E[AUTH]', 'To': 'haasdijk e[Author]'}], 'QueryTranslation': 'haasdijk e[Author]'}

Note how the result being produced is not in Python's native string format:

In [7]:
type( example_search['IdList'][0] )

Bio.Entrez.Parser.StringElement

The part of the query's result we are most interested in is accessible through

In [8]:
example_ids = [ int(id) for id in example_search['IdList'] ]
print(example_ids)

[39325889, 38631313, 33501027, 33501026, 33500899, 29311830, 28513205, 28513201, 28323435, 28140628, 26933487, 24977986, 24901702, 24852945, 24708899, 24252306, 23580075, 23144668, 22174697, 22154920]


### PubMed IDs dataset

We will now assemble a dataset comprised of research articles containing the given keyword, in either their titles or abstracts.

In [9]:
#  Due to the new limit (only 10K max results) the Entrez library is not suitable to fetch all ids for a query
#  - Therefore this has to be done via the more complicated way using the Entrez Direct e-utility:
#     https://www.ncbi.nlm.nih.gov/sites/books/NBK179288/ 
#  - also an API key has to be registered at NCBI
#  - the command used for fetching the ids: 
#               esearch -db pubmed -query "recognition" | efetch -format uid >results.txt
#  - thus, instead of using the output from the script below (recognition_ids.pkl.bz2) we use
search_term = 'recognition'

In [114]:
Ids_zip = 'data/' + search_term + '_Ids.zip'
Ids_file = search_term + '_Ids.txt'

if os.path.exists( Ids_zip ):
    with zipfile.ZipFile(Ids_zip, 'r') as myzip:
        
        with myzip.open(Ids_file) as myfile:
           Ids_str = myfile.read().decode('UTF-8').splitlines(keepends=True)

        # convert Ids to integers (and ensure that the conversion is reversible)
        Ids = [ int(id) for id in Ids_str ]
else:
     raise Exception('File not found' )



In [124]:
ids_to_remove = [
    39295396
]
for id_to_remove in ids_to_remove:
    try:
        Ids.remove(id_to_remove)
    except ValueError:
        print(f"ID {id_to_remove} not found in the list.")

    
    # Save list of Ids
    pickle.dump( Ids, bz2.BZ2File( Ids_file, 'wb' ) )
    
total = len( Ids )
print('%d documents contain the search term "%s".' % ( total, search_term ) )

481216 documents contain the search term "recognition".


Taking a look at what we just retrieved, here are the last 5 elements of the `Ids` list:

In [125]:



Ids[:5]

[53, 349, 480, 1346, 1635]

## ESummary: Retrieving summaries from primary IDs

To have a look at the kind of metadata we get from a call to `Entrez.esummary()`, we now fetch the summary of one of Haasdijk's papers (using one of the PubMed IDs we obtained in the previous section:

In [126]:
example_paper = Entrez.read( Entrez.esummary(db="pubmed", id='1310895') )[0]

def print_dict( p ):
    for k,v in p.items():
        print(k)
        print('\t', v)

print_dict(example_paper)

Item
	 []
Id
	 1310895
PubDate
	 1992 Feb 7
EPubDate
	 
Source
	 Cell
AuthorList
	 ['Bölker M', 'Urban M', 'Kahmann R']
LastAuthor
	 Kahmann R
Title
	 The a mating type locus of U. maydis specifies cell signaling components.
Volume
	 68
Issue
	 3
Pages
	 441-50
LangList
	 ['English']
NlmUniqueID
	 0413066
ISSN
	 0092-8674
ESSN
	 
PubTypeList
	 ['Journal Article']
RecordStatus
	 PubMed - indexed for MEDLINE
PubStatus
	 ppublish
ArticleIds
	 {'pubmed': ['1310895'], 'medline': [], 'doi': '10.1016/0092-8674(92)90182-c', 'pii': '0092-8674(92)90182-C'}
DOI
	 10.1016/0092-8674(92)90182-c
History
	 {'pubmed': ['1992/02/07 00:00'], 'medline': ['1992/02/07 00:01'], 'entrez': '1992/02/07 00:00'}
References
	 []
HasAbstract
	 IntegerElement(1, attributes={})
PmcRefCount
	 IntegerElement(0, attributes={})
FullJournalName
	 Cell
ELocationID
	 
SO
	 1992 Feb 7;68(3):441-50


For now, we'll keep just some basic information for each paper: title, list of authors, publication year, and [DOI](https://en.wikipedia.org/wiki/Digital_object_identifier).

In case you are not familiar with the DOI system, know that the paper above can be accessed through the link  `https://www.doi.org/` followed by the paper's DOI.

In [127]:
( example_paper['Title'], example_paper['AuthorList'], int(example_paper['PubDate'][:4]), example_paper['DOI'] )

('The a mating type locus of U. maydis specifies cell signaling components.',
 ['Bölker M', 'Urban M', 'Kahmann R'],
 1992,
 '10.1016/0092-8674(92)90182-c')

### Summaries dataset

We are now ready to assemble a dataset containing the summaries of all the paper `Ids` we previously fetched.

To reduce the memory footprint, and to ensure the saved datasets won't depend on Biopython being installed to be properly loaded, values returned by `Entrez.read()` will be converted to their corresponding native Python types. We start by defining a function for helping with the conversion of strings:

In [128]:
Summaries_file = 'data/' + search_term + '_Summaries.pkl.bz2'


In [129]:
if os.path.exists( Summaries_file ):
    Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )
    
else:
    # `Summaries` will be incrementally assembled, by performing multiple queries,
    # each returning at most `retrieve_per_query` entries.
    Summaries = []
    retrieve_per_query = 500
    
    print('Fetching Summaries of results: ')
    for start in range( 0, len(Ids), retrieve_per_query ):
        if (start % 10000 == 0):
            print('')
            print(start, end='')
            SummariesDict = dict( Summaries )
            print('length=', len(SummariesDict))
        else:
            print('.', end='')
        
        # build comma separated string with the ids at indexes [start, start+retrieve_per_query)
        query_ids = ','.join( [ str(id) for id in Ids[ start : start+retrieve_per_query ] ] )
       # print(query_ids)
        
        s = Entrez.read( Entrez.esummary( db="pubmed", id=query_ids ) )
        
        # out of the retrieved data, we will keep only a tuple (title, authors, year, DOI), associated with the paper's id.
        # (all values converted to native Python formats)
        for p in s:
            
            try:
                f = [
                    ( int( p['Id'] ), (
                        str( p['Title'] ),
                        [ str(a) for a in p['AuthorList'] ],
                        int( p['PubDate'][:4] ),                # keeps just the publication year
                        str( p.get('DOI', '') )            # papers for which no DOI is available get an empty string in their place
                        ) )
                    ]
                #print(int( p['Id'] ))
                Summaries.extend( f )
            except ValueError as e:
                print("\nError with ID " + p['Id'] + ": " + str(e))
                print("Manually remove this ID above and re-run code.")
    
    # Save Summaries, as a dictionary indexed by Ids
    Summaries = dict( Summaries )
    
    pickle.dump( Summaries, bz2.BZ2File( Summaries_file, 'wb' ) )


Fetching Summaries of results: 

0length= 0
...................
10000length= 10000
...................
20000length= 20000
...................
30000length= 30000
...................
40000length= 40000
...................
50000length= 50000
...................
60000length= 60000
...................
70000length= 70000
...................
80000length= 80000
...................
90000length= 90000
...................
100000length= 100000
...................
110000length= 110000
...................
120000length= 120000
...................
130000length= 130000
...................
140000length= 140000
...................
150000length= 150000
...................
160000length= 160000
...................
170000length= 170000
...................
180000length= 180000
...................
190000length= 190000
...................
200000length= 200000
...................
210000length= 210000
...................
220000length= 220000
...................
230000length= 230000
...................
240000lengt

Let us take a look at the first 3 retrieved summaries:

In [130]:
print(len(Ids))

print(Summaries[Ids[200001]])
print(len(Summaries))
{ id : Summaries[id] for id in Ids[1:3] }

481216
('Recovery of surface pose from texture orientation statistics under perspective projection.', ['Warren PA', 'Mamassian P'], 2010, '10.1007/s00422-010-0389-3')
481216


{349: ('Acute renal failure.', ['Freedman P', 'Smith EC'], 1975, ''),
 480: ('Recognition and significance of maternogenic fetal acidosis during intensive monitoring of labor.',
  ['Roversi GD', 'Canussio V', 'Spennacchio M'],
  1975,
  '10.1515/jpme.1975.3.1.53')}

## EFetch: Downloading full records from Entrez

`Entrez.efetch()` is the function that will allow us to obtain paper abstracts. Let us start by taking a look at the kind of data it returns when we query PubMed's database.

In [131]:
q = Entrez.read( Entrez.efetch(db="pubmed", id='349', retmode="xml") )['PubmedArticle']

`q` is a list, with each member corresponding to a queried id. Because here we only queried for one id, its results are then in `q[0]`.

In [132]:
type(q), len(q)

(list, 1)

1
At `q[0]` we find a dictionary containing two keys, the contents of which we print below.

In [133]:
type(q[0]), q[0].keys()

(Bio.Entrez.Parser.DictionaryElement,
 dict_keys(['MedlineCitation', 'PubmedData']))

In [134]:
print_dict( q[0][ 'PubmedData' ] )

ReferenceList
	 []
History
	 [DictElement({'Year': '1975', 'Month': '11', 'Day': '1'}, attributes={'PubStatus': 'pubmed'}), DictElement({'Year': '1975', 'Month': '11', 'Day': '1', 'Hour': '0', 'Minute': '1'}, attributes={'PubStatus': 'medline'}), DictElement({'Year': '1975', 'Month': '11', 'Day': '1', 'Hour': '0', 'Minute': '0'}, attributes={'PubStatus': 'entrez'})]
PublicationStatus
	 ppublish
ArticleIdList
	 [StringElement('349', attributes={'IdType': 'pubmed'})]


The key `'MedlineCitation'` maps into another dictionary. In that dictionary, most of the information is contained under the key `'Article'`. To minimize the clutter, below we show the contents of `'MedlineCitation'` excluding its `'Article'` member, and below that we then show the contents of `'Article'`.

In [135]:
print_dict( { k:v for k,v in q[0][ 'MedlineCitation' ].items() if k!='Article' } )

SpaceFlightMission
	 []
OtherID
	 []
OtherAbstract
	 []
GeneralNote
	 []
CitationSubset
	 ['IM']
KeywordList
	 []
InvestigatorList
	 []
PMID
	 349
DateCompleted
	 {'Year': '1976', 'Month': '02', 'Day': '13'}
DateRevised
	 {'Year': '2010', 'Month': '11', 'Day': '18'}
MedlineJournalInfo
	 {'Country': 'United States', 'MedlineTA': 'Heart Lung', 'NlmUniqueID': '0330057', 'ISSNLinking': '0147-9563'}
MeshHeadingList
	 [{'QualifierName': [StringElement('etiology', attributes={'UI': 'Q000209', 'MajorTopicYN': 'N'}), StringElement('metabolism', attributes={'UI': 'Q000378', 'MajorTopicYN': 'N'}), StringElement('therapy', attributes={'UI': 'Q000628', 'MajorTopicYN': 'N'})], 'DescriptorName': StringElement('Acute Kidney Injury', attributes={'UI': 'D058186', 'MajorTopicYN': 'Y'})}, {'QualifierName': [], 'DescriptorName': StringElement('Adolescent', attributes={'UI': 'D000293', 'MajorTopicYN': 'N'})}, {'QualifierName': [], 'DescriptorName': StringElement('Adult', attributes={'UI': 'D000328', 'MajorT

In [136]:
print_dict( q[0][ 'MedlineCitation' ][ 'Article' ] )

ELocationID
	 []
Language
	 ['eng']
ArticleDate
	 []
Journal
	 {'ISSN': StringElement('0147-9563', attributes={'IssnType': 'Print'}), 'JournalIssue': DictElement({'Volume': '4', 'Issue': '6', 'PubDate': {'Year': '1975', 'Season': 'Nov-Dec'}}, attributes={'CitedMedium': 'Print'}), 'Title': 'Heart & lung : the journal of critical care', 'ISOAbbreviation': 'Heart Lung'}
ArticleTitle
	 Acute renal failure.
Pagination
	 {'StartPage': '873', 'EndPage': '878', 'MedlinePgn': '873-8'}
Abstract
	 {'AbstractText': ['Acute renal failure is a life-threatening situation for which satisfactory treatment is currently available. Death is usually related to concomitant infection rather than uremia. Prompt recognition and treatment of underlying factors that predispose to renal insufficiency can frequently prevent the development of intrinsic acute renal failure.']}
AuthorList
	 ListElement([DictElement({'AffiliationInfo': [], 'Identifier': [], 'LastName': 'Freedman', 'ForeName': 'P', 'Initials': 'P'}, a

A paper's abstract can therefore be accessed with:

In [137]:
{ int(q[0]['MedlineCitation']['PMID']) : str(q[0]['MedlineCitation']['Article']['Abstract']['AbstractText'][0]) }

{349: 'Acute renal failure is a life-threatening situation for which satisfactory treatment is currently available. Death is usually related to concomitant infection rather than uremia. Prompt recognition and treatment of underlying factors that predispose to renal insufficiency can frequently prevent the development of intrinsic acute renal failure.'}

Some of the ids in our dataset refer to books from the [NCBI Bookshelf](http://www.ncbi.nlm.nih.gov/books/), a collection of freely available, downloadable, on-line versions of selected biomedical books. For such ids, `Entrez.efetch()` returns a slightly different structure, where the keys `[u'BookDocument', u'PubmedBookData']` take the place of the `[u'MedlineCitation', u'PubmedData']` keys we saw above.

### Abstracts dataset

We can now assemble a dataset mapping paper ids to their abstracts.

In [138]:
Abstracts_file = 'data/' + search_term + '_Abstracts.pkl.bz2'

In [139]:
import http.client
from collections import deque
from xml.dom import minidom
import re

def ch(node, childtype):
    return node.getElementsByTagName(childtype)[0]

if os.path.exists( Abstracts_file ):
    Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )
else:
    # `Abstracts` will be incrementally assembled, by performing multiple queries,
    # each returning at most `retrieve_per_query` entries.
    Abstracts = deque()
    retrieve_per_query = 500
    
    print('Fetching Abstracts of results: ')
    for start in range( 0, len(Ids), retrieve_per_query ):
        if (start % 10000 == 0):
            print('')
            print(start, end='')
        else:
            print('.', end='')
        
        # build comma separated string with the ids at indexes [start, start+retrieve_per_query)
        query_ids = ','.join( [ str(id) for id in Ids[ start : start+retrieve_per_query ] ] )
        
        # issue requests to the server, until we get the full amount of data we expect
        while True:
            try:
                #s = Entrez.read( Entrez.efetch(db="pubmed", id=query_ids, retmode="xml" ) )['PubmedArticle']
                s = minidom.parse( Entrez.efetch(db="pubmed", id=query_ids, retmode="xml" ) ).getElementsByTagName("PubmedArticle")
            except http.client.IncompleteRead:
                print('r', end='')
                continue
            break
        
        i = 0
        for p in s:
            abstr = ''
            if (p.getElementsByTagName('MedlineCitation')):
                citNode = ch(p,'MedlineCitation')
                pmid = ch(citNode,'PMID').firstChild.data
                articleNode = ch(citNode,'Article')
                if (articleNode.getElementsByTagName('Abstract')):
                    try:
                        abstr = ch(ch(articleNode,'Abstract'),'AbstractText').firstChild.data
                    except AttributeError:
                        abstr = ch(ch(articleNode,'Abstract'),'AbstractText').toprettyxml("  ")
                        abstr = re.sub(r"\s+", " ", re.sub("<[^>]*>", "", abstr))
            elif (p.getElementsByTagName('BookDocument')):
                bookNode = ch(p,'BookDocument')
                pmid = ch(bookNode,'PMID').firstChild.data
                if (bookNode.getElementsByTagName('Abstract')):
                    try:
                        abstr = ch(ch(bookNode,'Abstract'),'AbstractText').firstChild.data
                    except AttributeError:
                        abstr = ch(ch(bookNode,'Abstract'),'AbstractText').toprettyxml("  ")
                        abstr = re.sub(r"\s+", " ", re.sub("<[^>]*>", "", abstr))
            else:
                raise Exception('Unrecognized record type, for id %d (keys: %s)' % (Ids[start+i], str(p.keys())) )
            
            Abstracts.append( (int(pmid), str(abstr)) )
            i += 1
    
    # Save Abstracts, as a dictionary indexed by Ids
    Abstracts = dict( Abstracts )
    
    pickle.dump( Abstracts, bz2.BZ2File( Abstracts_file, 'wb' ) )

Fetching Abstracts of results: 

0...................
10000...................
20000...................
30000...................
40000...................
50000...................
60000...................
70000...................
80000...................
90000...................
100000...................
110000...................
120000...................
130000...................
140000...................
150000...................
160000...................
170000...................
180000...................
190000...................
200000...................
210000...................
220000...................
230000...................
240000...................
250000...................
260000...................
270000...................
280000...................
290000...................
300000...................
310000...................
320000...................
330000...................
340000...................
350000...................
360000...................
370000.............

Taking a look at one paper's abstract:

In [143]:
print(Abstracts[349])
print(len(Abstracts))
print(Ids[349])
dict_items_list = list(Abstracts.items())
print('------',dict_items_list[9990:101002])

Acute renal failure is a life-threatening situation for which satisfactory treatment is currently available. Death is usually related to concomitant infection rather than uremia. Prompt recognition and treatment of underlying factors that predispose to renal insufficiency can frequently prevent the development of intrinsic acute renal failure.
480451
79763


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



## ELink: Searching for related items in NCBI Entrez

To understand how to obtain paper citations with Entrez, we will first assemble a small set of PubMed IDs, and then query for their citations.
To that end, we search here for papers published in the Nature journal with our given keyword.

In [144]:
CA_search_term = search_term+'[TIAB] AND Nature[JOUR]'
CA_ids = Entrez.read( Entrez.esearch( db="pubmed", term=CA_search_term ) )['IdList']
CA_ids

['39415022', '39385031', '39385025', '39358508', '39232164', '39143217', '39143211', '39111359', '39085614', '39048818', '39020174', '39020165', '38961301', '38961291', '38961289', '38840020', '38839966', '38776963', '38750366', '38750358']

In [145]:
CA_summ = {
    p['Id'] : ( p['Title'], p['AuthorList'], p['PubDate'][:4], p['FullJournalName'], p.get('DOI', '') )
    for p in Entrez.read( Entrez.esummary(db="pubmed", id=','.join( CA_ids )) )
    }
CA_summ

{'39415022': ('A bacterial immunity protein directly senses two disparate phage proteins.',
  ['Zhang T', 'Cepauskas A', 'Nadieina A', 'Thureau A', "Coppieters 't Wallant K", 'Martens C', 'Lim DC', 'Garcia-Pino A', 'Laub MT'],
  '2024',
  'Nature',
  '10.1038/s41586-024-08039-y'),
 '39385031': ('A modular circuit coordinates the diversification of courtship strategies.',
  ['Coleman RT', 'Morantte I', 'Koreman GT', 'Cheng ML', 'Ding Y', 'Ruta V'],
  '2024',
  'Nature',
  '10.1038/s41586-024-08028-1'),
 '39385025': ('Structural basis of mRNA decay by the human exosome-ribosome supercomplex.',
  ['Kögel A', 'Keidel A', 'Loukeri MJ', 'Kuhn CC', 'Langer LM', 'Schäfer IB', 'Conti E'],
  '2024',
  'Nature',
  '10.1038/s41586-024-08015-6'),
 '39358508': ('Rapid homologue juxtaposition during meiotic chromosome pairing.',
  ['Nozaki T', 'Weiner B', 'Kleckner N'],
  '2024',
  'Nature',
  '10.1038/s41586-024-07999-5'),
 '39232164': ('A pathology foundation model for cancer diagnosis and prognosi

Because we restricted our search to papers in an open-access journal, you can then follow their DOIs to freely access their PDFs at the journal's website.

We will now issue calls to `Entrez.elink()` using these PubMed IDs, to retrieve the IDs of papers that cite them.
The database from which the IDs will be retrieved is [PubMed Central](http://www.ncbi.nlm.nih.gov/pmc/), a free digital database of full-text scientific literature in the biomedical and life sciences.

A complete list of the kinds of links you can retrieve with `Entrez.elink()` can be found [here](http://eutils.ncbi.nlm.nih.gov/entrez/query/static/entrezlinks.html).

In [147]:
CA_citing = {
    id : Entrez.read( Entrez.elink(
            cmd = "neighbor",               # ELink command mode: "neighbor", returns
                                            #     a set of UIDs in `db` linked to the input UIDs in `dbfrom`.
            dbfrom = "pubmed",              # Database containing the input UIDs: PubMed
            db = "pmc",                     # Database from which to retrieve UIDs: PubMed Central
            LinkName = "pubmed_pmc_refs",   # Name of the Entrez link to retrieve: "pubmed_pmc_refs", gets
                                            #     "Full-text articles in the PubMed Central Database that cite the current articles"
            from_uid = id                   # input UIDs
            ) )
    for id in CA_ids
    }

In [157]:
CA_citing['349']

KeyError: '349'

We have in `CA_citing[paper_id][0]['LinkSetDb'][0]['Link']` the list of papers citing `paper_id`. To get it as just a list of ids, we can do

In [153]:
cits = [ l['Id'] for l in CA_citing['39020165'][0]['LinkSetDb'][0]['Link'] ]
cits

['11476048',
 '11417361',
 '11411926',
 '11389603',
 '11364771',
 '11071384',
 '10557695']

However, one more step is needed, as what we have now are PubMed Central IDs, and not PubMed IDs. Their conversion can be achieved through an additional call to `Entrez.elink()`:

In [154]:
cits_pm = Entrez.read( Entrez.elink( dbfrom="pmc", db="pubmed", LinkName="pmc_pubmed", from_uid=",".join(cits)) )
cits_pm

[{'LinkSetDbHistory': [], 'ERROR': [], 'LinkSetDb': [{'Link': [{'Id': '39215040'}, {'Id': '39149900'}, {'Id': '38712175'}, {'Id': '37808811'}], 'DbTo': 'pubmed', 'LinkName': 'pmc_pubmed'}], 'DbFrom': 'pmc', 'IdList': ['11476048', '11417361', '11411926', '11389603', '11364771', '11071384', '10557695']}]

In [155]:
ids_map = { pmc_id : link['Id'] for (pmc_id,link) in zip(cits_pm[0]['IdList'], cits_pm[0]['LinkSetDb'][0]['Link']) }
ids_map

TypeError: 'ZipFile' object is not callable

And to check these papers:

In [50]:
{   p['Id'] : ( p['Title'], p['AuthorList'], p['PubDate'][:4], p['FullJournalName'], p.get('DOI', '') )
    for p in Entrez.read( Entrez.esummary(db="pubmed", id=','.join( ids_map.values() )) )
    }

{'39215040': ('Tail assembly interference is a common strategy in bacterial antiviral defenses.',
  ['He L', 'Miguel-Romero L', 'Patkowski JB', 'Alqurainy N', 'Rocha EPC', 'Costa TRD', 'Fillol-Salom A', 'Penadés JR'],
  '2024',
  'Nature communications',
  '10.1038/s41467-024-51915-4'),
 '39149900': ('A phage satellite manipulates the viral DNA packaging motor to inhibit phage and promote satellite spread.',
  ['Boyd CM', 'Seed KD'],
  '2024',
  'Nucleic acids research',
  '10.1093/nar/gkae675'),
 '37808811': ('Bacterial antiviral defense pathways encode eukaryotic-like ubiquitination systems.',
  ['Chambers LR', 'Ye Q', 'Cai J', 'Gong M', 'Ledvina HE', 'Zhou H', 'Whiteley AT', 'Suhandynata RT', 'Corbett KD'],
  '2023',
  'bioRxiv : the preprint server for biology',
  '10.1101/2023.09.26.559546')}

### Citations dataset

We have now seen all the steps required to assemble a dataset of citations to each of the papers in our dataset.

In [175]:
Citations_file = 'data/' + search_term + '_Citations.pkl.bz2'
Citations = []

At least one server query will be issued per paper in `Ids`. Because NCBI allows for at most 3 queries per second (see [here](http://biopython.org/DIST/docs/api/Bio.Entrez-pysrc.html#_open)), this dataset will take a long time to assemble. Should you need to interrupt it for some reason, or the connection fail at some point, it is safe to just rerun the cell below until all data is collected.

In [188]:
import http.client
from urllib.error import HTTPError
if Citations == [] and os.path.exists( Citations_file ):
    Citations = pickle.load( bz2.BZ2File( Citations_file, 'rb' ) )

if len(Citations) < len(Ids):
    
    i = len(Citations)
    checkpoint = int(len(Ids) / 1000) + 1      # save to hard drive at every 0.1% of Ids fetched
    
    for pm_id in Ids[i:]:               # either starts from index 0, or resumes from where we previously left off
        
        while True:
            try:
                # query for papers archived in PubMed Central that cite the paper with PubMed ID `pm_id`
                c = Entrez.read( Entrez.elink( dbfrom = "pubmed", db="pmc", LinkName = "pubmed_pmc_refs", id=str(pm_id) ) )
                
                c = c[0]['LinkSetDb']
                if len(c) == 0:
                    # no citations found for the current paper
                    c = []
                    #print("no links for",pm_id)
                else:
                    c = [ l['Id'] for l in c[0]['Link'] ]
                    
                    # convert citations from PubMed Central IDs to PubMed IDs
                    p = []
                    retrieve_per_query = 100
                    for start in range( 0, len(c), retrieve_per_query ):
                        query_ids = ','.join( c[start : start+retrieve_per_query] )
                        r = Entrez.read( Entrez.elink( dbfrom="pmc", db="pubmed", LinkName="pmc_pubmed", from_uid=query_ids ) )
                        # select the IDs. If no matching PubMed ID was found, [] is returned instead
                        p.extend( [] if r[0]['LinkSetDb']==[] else [ int(link['Id']) for link in r[0]['LinkSetDb'][0]['Link'] ] )
                    c = p
            
            except http.client.BadStatusLine:
                # Presumably, the server closed the connection before sending a valid response. Retry until we have the data.
                print('r')
                continue
            except http.client.IncompleteRead:
                print('incompleteRead, continuing')
                continue
            except HTTPError:
                print('r')
                continue
            break
        
        Citations.append( (pm_id, c) )
       # print(pm_id,' :',c)
        if (i % 1000 == 0):
            print('')
            print(i, end='')
        if (i % 10 == 0):
            print('.', end='')
        i += 1
        
        if i % checkpoint == 0:
            print('\tsaving at checkpoint', i)
            pickle.dump( Citations, bz2.BZ2File( Citations_file, 'wb' ) )
    
    print('\n done.')
    
    # Save Citations, as a dictionary indexed by Ids
    Citations = dict( Citations )
    
    pickle.dump( Citations, bz2.BZ2File( Citations_file, 'wb' ) )

..........................	saving at checkpoint 69890
...........
70000........r
..............................	saving at checkpoint 70372
................................................	saving at checkpoint 70854
..............
71000..................................	saving at checkpoint 71336
................................................	saving at checkpoint 71818
..................
72000..............................	saving at checkpoint 72300
.................................................	saving at checkpoint 72782
.....................
73000...........................	saving at checkpoint 73264
................................................	saving at checkpoint 73746
.........................
74000.......................	saving at checkpoint 74228
................................................	saving at checkpoint 74710
.............................
75000....................	saving at checkpoint 75192
................................................	saving at checkpoint

KeyboardInterrupt: 

To see that we have indeed obtained the data we expected, you can match the ids below, with the ids listed at the end of last section.

In [198]:
print(len(Citations))
Citations[84842]
Summaries[10576728]


84843


('The male determinant of self-incompatibility in Brassica.',
 ['Schopfer CR', 'Nasrallah ME', 'Nasrallah JB'],
 1999,
 '10.1126/science.286.5445.1697')

## Where do we go from here?

Running the code above generates multiple local files, containing the datasets we'll be working with. Loading them into memory is a matter of just issuing a call like<br>
``data = pickle.load( bz2.BZ2File( data_file, 'rb' ) )``.

The Entrez module will therefore no longer be needed, unless you wish to extend your data processing with additional information retrieved from PubMed.

Should you be interested in looking at alternative ways to handle the data, have a look at the [sqlite3](http://docs.python.org/3/library/sqlite3.html) module included in Python's standard library, or [Pandas](http://pandas.pydata.org/), the Python Data Analysis Library.